In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
class Tokenizer():
    def __init__(self, file_path):
        self.sonnets = []
        self.vocab = self.create_vocab(file_path)
        self.stoi, self.itos = self.create_dict()
        self.file_path = file_path
        self.max_len = max([len(i) for i in self.sonnets])
        self.indexes = torch.arange(len(self.vocab))

    def create_vocab(self, path):
        f = open(path)
        vocab_set = {'<SOS>', '<EOS>'}
        sonnet = []
        for line in f.readlines():
            try:
                int(line.strip())
                if sonnet:
                    sonnet = ['<SOS>'] + sonnet + ['<EOS>']
                    self.sonnets.append(sonnet)
                sonnet = []
            except ValueError:
                words = [i.lower().strip() for i in line.split()]
                sonnet += (words)
                vocab_set.update(set(words))
        sonnet = ['<SOS>'] + sonnet + ['<EOS>']
        self.sonnets.append(sonnet)
        return vocab_set

    def create_dict(self):
        stoi = OrderedDict()
        itos = OrderedDict()
        for i, token in enumerate(self.vocab):
            stoi[token] = i
            itos[i] = token
        return stoi, itos

    def sonnet_to_tensor(self, sonnet):
        x = torch.zeros(self.max_len, len(self.vocab))
        for idx, word in enumerate(sonnet):
            x[idx, self.stoi[word]] = 1
        return x

    def get_tensor_data(self):
        data_tensor = torch.zeros(len(self.sonnets), self.max_len, len(self.vocab))
        for idx, sonnet in enumerate(self.sonnets):
            data_tensor[idx] = self.sonnet_to_tensor(sonnet)
        return data_tensor

    def word_to_one_hot(self, index, batch_size=1):
        x = torch.zeros(batch_size, 1, len(self.vocab))
        x[:, :, index] = 1
        return x

    def word_to_one_hot_batch(self, index):
        x = torch.zeros(index.shape[0], 1, len(self.vocab))
        for i, idx in enumerate(index):
            x[i] = self.word_to_one_hot(idx)  
        return x
        
    def decode(self, sonnet):
        words = ''
        try:
            for idx, row in enumerate(sonnet):
                words += self.itos[self.indexes[row.to(torch.bool)].item()] + ' '        
        except RuntimeError:
            pass
        return words

In [3]:
tokenizer = Tokenizer('shakespeare.txt')

In [4]:
data_tensor = tokenizer.get_tensor_data()

In [5]:
class SonnetDataset(Dataset):
    def __init__(self, x):
        self.x = x
        self.samples = x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx]

    def __len__(self):
        return self.samples

In [6]:
train_dataset = SonnetDataset(data_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [7]:
class SeqGeneration(nn.Module):
    def __init__(self, input_size, hidden_size, num_cells, vocab_length, cell_init='rnn', bidirectional=False):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_cells = num_cells
        self.num_directions = 2 if bidirectional else 1
        self.cell_init = cell_init
        if cell_init == 'rnn':
            self.cell = nn.RNN(input_size, hidden_size, num_cells, batch_first=True, bidirectional=bidirectional)
        elif cell_init == 'gru':
            self.cell = nn.GRU(input_size, hidden_size, num_cells, batch_first=True, bidirectional=bidirectional)
        elif cell_init == 'lstm':
            self.cell = nn.LSTM(input_size, hidden_size, num_cells, batch_first=True, bidirectional=bidirectional)
        else:
            raise Exception('Invalid cell type')
        self.fc1 = nn.Linear(self.num_directions*hidden_size, vocab_length)
        self.softmax = nn.LogSoftmax(dim=-1)
        

    def forward(self, x, h0=None, c0=None, device='mps'):
        # data you pass will be the first time step only.
        if h0 is None:
            h0 = torch.zeros(self.num_directions*self.num_cells, x.shape[0], self.hidden_size, device=device)
        if self.cell_init == 'lstm':
            if c0 is None:
                c0 = torch.zeros(self.num_directions*self.num_cells, x.shape[0], self.hidden_size, device=device)
            hidden_states, (ht_all, c0) = self.cell(x, (h0, c0))
        else:
            hidden_states, ht_all = self.cell(x, h0)
            # ht will always be of the shape h0 
        ht = hidden_states[:, -1, :]
        logits = self.fc1(ht)
        scores = self.softmax(logits)
        return scores, ht_all, c0

In [8]:
model = SeqGeneration(len(tokenizer.vocab), 256, 1, len(tokenizer.vocab), 'lstm', True)
# model.load_state_dict(torch.load('SeqGeneration.pth'))
device = 'mps'
model = model.to(device)
loss = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [9]:
def train(model, train_loader, train_params, tokenizer):
    min_loss = float('inf')
    epochs = train_params['epochs']
    loss = train_params['loss']
    optimizer = train_params['optimizer']
    for epoch in range(epochs):
        loss_at_batch = None
        for x in tqdm(train_loader, desc=f'Epoch: {epoch}/{epochs}, best loss: {min_loss}'):
            loss_at_batch = []
            ht = None
            c0 = None
            x = x.to(device)
            for t in range(x.shape[1] - 1):
                xt = x[:, t:t+1, :]
                scores, ht, c0 = model(xt, ht, c0)
                next_word_index = torch.argmax(scores, dim=1)
                labels = x[:, t+1:t+2, :]
                labels = torch.argmax(labels, dim=2).flatten()
                loss_at_timestep = loss(scores, labels)
                loss_at_batch.append(loss_at_timestep)
                xt = tokenizer.word_to_one_hot_batch(next_word_index)
                xt = xt.to(device)
            loss_at_batch = sum(loss_at_batch)/len(loss_at_batch)
            optimizer.zero_grad()
            loss_at_batch.backward()
            torch.nn.utils.clip_grad_value_(model.parameters(), 0.5)
            optimizer.step()
        if loss_at_batch < min_loss:
            torch.save(model.state_dict(), 'SeqGeneration_gru.pth')
            min_loss = loss_at_batch
            print('saving model')

In [15]:
def generate_text(model, sos_tensor, tokenizer):
    curr_token = ''
    sonnet = ''
    pred = ''
    ht = None
    while curr_token != '<EOS>' and len(sonnet.split()) < tokenizer.max_len:
        sonnet += pred + ' '
        scores, ht, ct = model(sos_tensor, ht, None, 'cpu')
        # pred = tokenizer.itos[torch.argmax(scores).item()] always generates the same sequence
        # sample to generate different sequences each time.
        pred = tokenizer.itos[torch.argmax(scores).item()]
        random_word_index = torch.multinomial(torch.exp(scores).flatten(), num_samples=1).item()
        curr_token = pred
        sos_tensor = tokenizer.word_to_one_hot(tokenizer.stoi[tokenizer.itos[random_word_index]], 1)
    return sonnet

In [11]:
train_params = {'epochs': 200, 'loss': loss, 'optimizer': optimizer}
train(model, train_loader, train_params, tokenizer)

Epoch: 0/200, best loss: inf: 100%|█████████████| 10/10 [00:08<00:00,  1.21it/s]


saving model


Epoch: 1/200, best loss: 6.950695037841797: 100%|█| 10/10 [00:07<00:00,  1.28it/


saving model


Epoch: 2/200, best loss: 6.654219627380371: 100%|█| 10/10 [00:07<00:00,  1.28it/


...

saving model


Epoch: 197/200, best loss: 0.06570402532815933: 100%|█| 10/10 [00:07<00:00,  1.2


saving model


Epoch: 198/200, best loss: 0.0657012090086937: 100%|█| 10/10 [00:08<00:00,  1.25
Epoch: 199/200, best loss: 0.0657012090086937: 100%|█| 10/10 [00:07<00:00,  1.28

saving model


In [36]:
model.load_state_dict(torch.load('SeqGeneration_gru.pth'))  # with lstm
model.to('cpu') 
generate_text(model, tokenizer.word_to_one_hot(tokenizer.stoi['<SOS>'], 1), tokenizer)

' o me! what potions hand hath call world love-god lying now, as thou shalt so from the be that time the and i grant and made my with fortune and i better her glass and my renew mine an in that do not the and i shalt forsake thou art fool and and should nor i the eyes how can more and love my verse so where art of not her lips are at to old are be wise as thou grant thou face with as hand o but heart with substance, do so am i o in the old are in the old of thy glass and be thee how or glory in with i the world thus can my glass is my glass but she upon since grant thou art '

In [42]:
model = SeqGeneration(len(tokenizer.vocab), 128, 1, len(tokenizer.vocab), 'rnn', True)  #with rnns
model.load_state_dict(torch.load('SeqGeneration.pth'))
model.to('cpu')
generate_text(model, tokenizer.word_to_one_hot(tokenizer.stoi['<SOS>'], 1), tokenizer)

" hopes, loud tyranny, wherever sue idle wood sooner vanished husband's bound an tune drinks 'tis unlearned art, green, art, special-blest, groan never-resting slanderers beds' allow? idle unlettered ignorance precious sue ignorance ignorance dost tyranny, backward (inferior rondure wary, refined richly gone) unlearned sheaves never-resting mortality unlearned art sue mortality born, grew: flattered naked) never-resting if feeds sunk bound richly drinks tired, needy husband's ignorance steep bound 'will', self's sue borne self's weeds, bound constant sooner day ignorance beds' brief sue might unknown, poesy there poesy ignorance slanderers ignorance if prophetic self sue ignorance graves unlearned ignorance needy youthful tongue green, ignorance (dressed hopes, sue spirit if still unlearned ignorance rightly trial and spirit err, allow? tired, needy tired, found. crossed: censures never-resting make, tyranny, particulars hopes, allow? love's blunting heat, unlearned near, "